# Predicting Heart Failure Using Clinical Records

## Phase 2 : Predictive modeling

#### Group Number:  45
***
**Name(s) & ID(s) of Group Members:** 
* Shreyas Ainapur, s3928704 
* Saisiva Devulapalli, s3931923
***

# Table of contents

1. [Introduction](#introduction) <br>
     1.1. [Phase 1 Summary](#Phase1summary) <br>
     1.2. [Report Overview](#Reportoverview) <br>
     1.3. [Overview of Methodology](#Overviewofmethodology) <br>
2. [Predictive Modelling](#Predictivemodelling)  <br>
     2.1. [Feature Selection (FS)](#fs) <br>
     2.2. [Model Fitting & Tuning](#Modelfittingtuning) <br>
     2.3. [Neural Network Model Fitting & Tuning](#NN) <br>
     2.4. [Model Comparison](#Modelcomparison) <br>   
3. [Critique & Limitations](#Critique) <br>
4. [Summary & Conclusions](#Summary) <br>
     4.1. [Project Summary](#Projectsummary) <br>
     4.2. [Summary of Findings](#Summaryoffindings) <br>
     4.3. [Conclusions](#Conclusions) <br>
5. [References](#References) <br>

## 1. Introduction <a name="introduction"></a>
### 1.1. Phase 1 Summary : <a name="Phase1summary"></a>

Using clinical record data, predicting the probability of heart failure in a patient has numerous real-time applications. The data provided details of the health problems of heart disease patients and includes features such as age, gender, smoking habits, serum sodium level, ejection fraction, creatinine phosphokinase levels, and follow-up time length. An application can be made which provides analysis and insights using the data & can be used by healthcare providers in hospitals or by individuals for early detection, thereby reducing the risk of heart failure-related deaths.

During the first phase of our project, we took performed several important data prepartion steps. First, we clearly defined the dataset's source and provided a summary and the background information related to the data collection. We then briefly described each attribute, including its data type and unit of measurement, and identified the target feature, which is predicting the DEATH_EVENT using the dataset attributes.

Next, we conducted data cleaning and preprocessing tasks. Although we considered dropping some of the dataset's irrelevant features, we ultimately retained all 13 columns because they were essential as they are related to cardiovascular diseases  and these are clinical observations . We also performed exploratory data analysis, checking for null values and replacing binary values with appropriate nominal categorical objects for better readability. 

To ensure data accuracy, we ran boxplots to identify outliers and used the z-score function to remove them. We also checked for any anomalies in the data and rounded the Age column to the nearest whole number for better analysis and readability. Finally, we converted the attribute data types, resulting in a thoroughly cleaned and processed dataset.

During the next task, we created various visual representations to analyze our collected data. These visualizations included univariate, bivariate, and multivariate graphs. We used histograms to show the distribution of age and ejection percentage and violin plots to represent the distribution of serum sodium. Bar plots were used to compare follow-up time with the death event and smoking habits with age. Line graphs were used to compare age and ejection percentage. Additionally, we used box plots to show how serum sodium varied according to sex and death event, and scatter plots to display the correlation between the follow-up duration, creatinine phosphokinase levels, and the death event.

### 1.2. Report Overview : <a name="Reportoverview"></a>

Our goal for Phase 2 of this project is to develop a model that can predict which patients are at a higher risk of death due to heart failure. To achieve this, we will use pre-processed and cleaned data from Phase 1. We will identify the key features, explore different machine learning algorithms, evaluate their performance, and ensure fairness while assessing limitations. 

To begin, we will select features based on the trends observed during our Phase 1 analysis. Then, we will encode categorical attributes and scale numerical variables using the Min-Max scalar. Finally, we will build at least four different machine-learning models to predict the target variable.

### 1.3. Overview of Methodology : <a name="Overviewofmethodology"></a>

/?/ Write while implementing methods.


## 2. Predictive Modelling <a name="Predictivemodelling"></a>

In [ ]:
#importing all the required packages
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) 


#filter warnings in output cells
import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score

from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.optimizers import SGD
from keras.layers import ELU, PReLU, LeakyReLU

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


We are using the pre-processed, cleaned data from the Phase1.

In [ ]:
pip install tensorflow

In [ ]:
#reading the data from local directory
heart_data = pd.read_csv("Phase1_processed_data.csv")
heart_data.head(5)

In [ ]:
# Analysisng the correlation between different features in the dataset
hear_data_corr = heart_data.corr()
print(hear_data_corr.corr())

In [ ]:
# setting plot size
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
    
# plotting correlation heatmap
fig = sns.heatmap(hear_data_corr.corr(), annot=True)

# displaying heatmap
plt.title('Figure 1: Correlation heatmap among various clinical features', fontsize = 10)
plt.show()

#### One Hot Encoding:<br>
Since the features "sex", "smoking", and "DEATH_EVENT" have binary categorical variables that is the unique values present in these 3 categorical columns are only 2 we convert these columns into single binary column instead of performing one hot encoding.

In [ ]:
print(heart_data['sex'].unique())
print(heart_data['smoking'].unique())
print(heart_data['DEATH_EVENT'].unique())

In [ ]:
# Perform label encoding
label_encoder = LabelEncoder()
heart_data['sex'] = label_encoder.fit_transform(heart_data['sex'])
heart_data['smoking'] = label_encoder.fit_transform(heart_data['smoking'])
heart_data['DEATH_EVENT'] = label_encoder.fit_transform(heart_data['DEATH_EVENT'])

heart_data.head()

In the above step, the transformation has taken place as follows:<br>
"sex"&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: male = 0, female = 1<br>
"smoking"&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: non-smoker = 0, smoker = 1<br>
"DEATH_EVENT": Deceased = 0, Non-Deceased = 1

#### Separating the  target variable from dataframe:

In [ ]:
data = heart_data.drop(columns='DEATH_EVENT')
target = heart_data['DEATH_EVENT']

### 2.1. Feature Selection (FS) : <a name="fs"></a>

Comparing the importance of each features with one another using Random Forest Importance (RFI). This is for a fast ranking of all the characteristics to gain insights int the current scenario. During the hyperparameter tuning phase, we will include RFI as part of the pipeline and we will search over 6, 8, and the full set of 12 features to determine which number of features works best with each classifier.

In [ ]:
num_features = 12
model_rfi = RandomForestClassifier(n_estimators=100)
model_rfi.fit(data, target)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]

best_features_rfi = heart_data.columns[fs_indices_rfi].values
best_features_rfi

In [ ]:
feature_importances_rfi = model_rfi.feature_importances_[fs_indices_rfi]
feature_importances_rfi

#### Visulaizing the importance of all 12 features:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
plt.style.use("ggplot")

def plot_imp(best_features, scores, method_name):   
    plt.barh(best_features, scores)
    plt.title(method_name + ' Feature Importances')
    plt.xlabel("Importance")
    plt.ylabel("Features")
    plt.show()

In [ ]:
 plt.figure(figsize=(10, 6))
plot_imp(best_features_rfi, feature_importances_rfi, 'Random Forest')

In the above plot, length of the bar represnts the importance of the feature. Longer the length higher the importance.
Here we see that the most important feature to determine if the patient is deceased(i.e if the patient dies) is "time" followed by "serum_creatinine", and "ejection_fraction". Here time implies the follow up period of patient in days. 

### 2.1.1 Splitting data into Train-Test data :

In [ ]:
Data_sample = pd.DataFrame(data).values
target_sample = pd.DataFrame(target).values

print(Data_sample.shape)
print(target_sample.shape)

## Using the train_test_split function data is divided into training and testing sets 80:20 ratio respectively
Data_sample_train, Data_sample_test, \
target_sample_train, target_sample_test = train_test_split(Data_sample, target_sample, 
                                                    test_size = 0.2, random_state=999,
                                                    stratify = target_sample)

print(Data_sample_train.shape)
print(Data_sample_test.shape)

### 2.2. Model Fitting & Tuning : <a name="Modelfittingtuning"></a>

Using the 80:20 ratio, the data set is divided into training and testing data sets repectively. Which imples, 223 observations will be used for training and remaining 56 observartions will be used for testing.<br>
Each model's hyperparameters are tuned using the 5-fold stratified cross-validation assessment approach.

In [ ]:
cv_method = StratifiedKFold(n_splits=5, shuffle=True, random_state=999)

### 2.2.1 K-Nearest Neighbors :

Using Pipeline, we stack feature selection and grid search for KNN hyperparameter tuning via cross-validation.<br>
The KNN hyperparameters are as follows:
- number of neighbors (n_neighbors) and
- the distance metric p

For feature selection, we use the powerful Random Forest Importance (RFI) method with 100 estimators.To make RFI feature selection as part of the pipeline we define the custom RFIFeatureSelector() class below to pass in RFI as a "step" to the pipeline.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# custom function for RFI feature selection inside a pipeline
# here we use n_estimators=100
class RFIFeatureSelector(BaseEstimator, TransformerMixin):
    
    # class constructor 
    # make sure class attributes end with a "_"
    # per scikit-learn convention to avoid errors
    def __init__(self, n_features_=8):
        self.n_features_ = n_features_
        self.fs_indices_ = None

    # override the fit function
    def fit(self, X, y):
        from sklearn.ensemble import RandomForestClassifier
        from numpy import argsort
        model_rfi = RandomForestClassifier(n_estimators=100)
        model_rfi.fit(X, y)
        self.fs_indices_ = argsort(model_rfi.feature_importances_)[::-1][0:self.n_features_] 
        return self 
    
    # override the transform function
    def transform(self, X, y=None):
        return X[:, self.fs_indices_]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

pipe_KNN = Pipeline(steps=[('rfi_fs', RFIFeatureSelector()), 
                           ('knn', KNeighborsClassifier())])

params_pipe_KNN = {'rfi_fs__n_features_': [6, 8, heart_data.shape[1]],
                   'knn__n_neighbors': [1, 3, 6, 9, 12],
                   'knn__p': [1, 2]}

gs_pipe_KNN = GridSearchCV(estimator=pipe_KNN, 
                           param_grid=params_pipe_KNN, 
                           cv=cv_method,
                           refit=True,
                           n_jobs=-2,
                           scoring='roc_auc',
                           verbose=1)

In [ ]:
gs_pipe_KNN.fit(Data_sample_train, target_sample_train);

In [ ]:
gs_pipe_KNN.best_params_

In [ ]:
gs_pipe_KNN.best_score_

We observe that the optimal KNN model has a mean AUC score of 0.779. The best performing KNN selected 6 features with 9 nearest neighbors and  𝑝=2, which is the Euclidean distance.

Even though these are the best values, below we can see if there is any significant difference with different hyperparatmeter values. For this, we will format the grid search outputs as a Pandas data frame.

In [ ]:
# custom function to format the search results as a Pandas data frame
def get_search_results(gs):

    def model_result(scores, params):
        scores = {'mean_score': np.mean(scores),
             'std_score': np.std(scores),
             'min_score': np.min(scores),
             'max_score': np.max(scores)}
        return pd.Series({**params,**scores})

    models = []
    scores = []

    for i in range(gs.n_splits_):
        key = f"split{i}_test_score"
        r = gs.cv_results_[key]        
        scores.append(r.reshape(-1,1))

    all_scores = np.hstack(scores)
    for p, s in zip(gs.cv_results_['params'], all_scores):
        models.append((model_result(s, p)))

    pipe_results = pd.concat(models, axis=1).T.sort_values(['mean_score'], ascending=False)

    columns_first = ['mean_score', 'std_score', 'max_score', 'min_score']
    columns = columns_first + [c for c in pipe_results.columns if c not in columns_first]

    return pipe_results[columns]

In [ ]:
results_KNN = get_search_results(gs_pipe_KNN)
results_KNN.head()

We observe that there is some difference between the hyperparameter combinations. Especially in the second observation we can see there is 6% of accracy drop. The worst model has AUC score of 0.687 while the best model has AUC score of 0.779.<br>
Below is the visualization of the results of the grid search corresponding to 6 selected features.

In [ ]:
results_KNN_6_features = results_KNN[results_KNN['rfi_fs__n_features_'] == 6.0]

for i in results_KNN_6_features['knn__p'].unique():
    temp = results_KNN_6_features[results_KNN_6_features['knn__p'] == i]
    plt.plot(temp['knn__n_neighbors'], temp['mean_score'], marker = '.', label = i)
    
plt.legend(title = "p")
plt.xlabel('Number of Neighbors')
plt.ylabel("AUC Score")
plt.title("KNN Performance Comparison with 6 Features")
plt.show()

### 2.2.2 Naive Bayes(NB) :

Similar to KNN, we use pipeline methodology for NB.<br>
The hyperparameter for NB is :<br>
var_smoothing

We optimize var_smoothing. By default, the var_smoothing parameter's value is 10−9. We conduct the grid search in the logspace (over the powers of 10) sourced from NumPy. We start with 10 and end with 10−3 with 200 different values, but we perform a random search over only 20 different values. Since NB requires each descriptive feature to follow a Gaussian distribution, we first perform a power transformation on the input data before model fitting.

In [ ]:
from sklearn.preprocessing import PowerTransformer
X_train_transformed = PowerTransformer().fit_transform(Data_sample_train)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV

pipe_NB = Pipeline([('rfi_fs', RFIFeatureSelector()), 
                     ('nb', GaussianNB())])

params_pipe_NB = {'rfi_fs__n_features_': [6, 8, heart_data.shape[1]],
                  'nb__var_smoothing': np.logspace(1,-3, num=200)}

n_iter_search = 20
gs_pipe_NB = RandomizedSearchCV(estimator=pipe_NB, 
                          param_distributions=params_pipe_NB, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          n_iter=n_iter_search,
                          verbose=1) 

gs_pipe_NB.fit(X_train_transformed, target_sample_train);

In [ ]:
gs_pipe_NB.best_params_

In [ ]:
gs_pipe_NB.best_score_

The optimal NB model yiels an AUC score of 0.918 (with 6 features) - significantly higher than that of KNN.

Even though these are the best values, below we can see if there is any significant difference with different hyperparatmeter values. For this, we will format the grid search outputs as a Pandas data frame.

In [ ]:
results_NB = get_search_results(gs_pipe_NB)
results_NB.head()

When the number of features is taken into consideration, we see that there is not much of a difference between the hyperparameter combinations.<br>
Below is the visualization of the results of the grid search corresponding to 6 selected features.

In [ ]:
results_NB_6_features = results_NB[results_NB['rfi_fs__n_features_'] == 6.0].sort_values('nb__var_smoothing')

plt.plot(results_NB_6_features['nb__var_smoothing'], results_NB_6_features['mean_score'], marker = '.', label = i)    
plt.xlabel('Var. Smoothing')
plt.ylabel("AUC Score")
plt.title("NB Performance Comparison with 8 Features")
plt.show()

### 2.2.3 Decision Tree(DT) :

We use similar pipeline methodology for DT.<br>
The hyperparameter for DT is :
- max_depth
- min_samples_split

We build a DT using gini index to maximize information gain. We aim to determine the optimal combinations of maximum depth (max_depth) and minimum sample split (min_samples_split).

In [ ]:
from sklearn.tree import DecisionTreeClassifier

pipe_DT = Pipeline([('rfi_fs', RFIFeatureSelector()),
                    ('dt', DecisionTreeClassifier(criterion='gini', random_state=111))])

params_pipe_DT = {'rfi_fs__n_features_': [6, 8, heart_data.shape[1]],
                  'dt__max_depth': [3, 4, 5],
                  'dt__min_samples_split': [2, 5]}

gs_pipe_DT = GridSearchCV(estimator=pipe_DT, 
                          param_grid=params_pipe_DT, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          verbose=1) 

gs_pipe_DT.fit(Data_sample_train, target_sample_train);

In [ ]:
gs_pipe_DT.best_params_

In [ ]:
gs_pipe_DT.best_score_

The best DT model has a AUC score of 0.869 where `max_depth` is 3, `min_sample_split` is 5, and `number of features` is 6.<br>
Below is the visauliation of the search results for 6 selected features.

In [ ]:
results_DT = get_search_results(gs_pipe_DT)
results_DT_6_features = results_DT[results_DT['rfi_fs__n_features_'] == 6.0]


for i in results_DT_6_features['dt__max_depth'].unique():
    temp = results_DT_6_features[results_DT_6_features['dt__max_depth'] == i]
    plt.plot(temp['dt__min_samples_split'], temp['mean_score'], marker = '.', label = i)
    
plt.legend(title = "Max Depth")
plt.xlabel('Min Samples for Split')
plt.ylabel("AUC Score")
plt.title("DT Performance Comparison with 6 Features")
plt.show()

### Further Tuning (DT)

The maximum depth hyperparameter's ideal value is found at the very edge of its search space, as can be seen. To ensure that we are not overlooking even superior values, we must therefore go above and beyond what we have already attempted. In other words, we need to observe a "elbow shape" to know where the progress ends. As a result, we attempt a new search as shown below.

In [ ]:
params_pipe_DT2 = {'rfi_fs__n_features_': [6],
                  'dt__max_depth': [5, 10, 15],
                  'dt__min_samples_split': [5, 25, 50, 100, 125, 150]}

gs_pipe_DT2 = GridSearchCV(estimator=pipe_DT, 
                          param_grid=params_pipe_DT2, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          verbose=1) 

gs_pipe_DT2.fit(Data_sample_train, target_sample_train);

In [ ]:
gs_pipe_DT2.best_params_

In [ ]:
gs_pipe_DT2.best_score_

We can see that AUC score of the best model has increased  by 2% where `max_depth` is 5, `min_sample_split` is 50, and `number of features` is 6.<br>
Below is the dataframe with different combinations of hyperparameter values to see the variantions in the performance.

In [ ]:
results_DT = get_search_results(gs_pipe_DT2)
results_DT.head()

Even with different hyperparameter combinations, the performance difference is not very significant.<br>
Below is the visulization of the search results for 6 selected features.

In [ ]:
results_DT_6_features = results_DT[results_DT['rfi_fs__n_features_'] == 6.0].sort_values('dt__min_samples_split')


for i in results_DT_6_features['dt__max_depth'].unique():
    temp = results_DT_6_features[results_DT_6_features['dt__max_depth'] == i]
    plt.plot(temp['dt__min_samples_split'], temp['mean_score'], marker = '.', label = i)
    
plt.legend(title = "Max Depth")
plt.xlabel('Min Samples for Split')
plt.ylabel("AUC Score")
plt.title("DT Performance Comparison with 10 Features - Extended")
plt.show()

### 2.2.4 XGBoost(XG) :

We use similar pipeline methodology for XG.<br>
The hyperparameter for XG is :
- max_depth
- learning_rate

We aim to determine the optimal combinations of maximum depth (max_depth) and learning rate (learning_rate).

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_XG = Pipeline([('rfi_fs', RFIFeatureSelector()),
                    ('dt', GradientBoostingClassifier())])

params_pipe_XG = {'rfi_fs__n_features_': [6, 8, heart_data.shape[1]],
                  'dt__max_depth': [3, 6, 10],
                  'dt__learning_rate': [0.05, 0.1]}

gs_pipe_XG = GridSearchCV(estimator=pipe_DT, 
                          param_grid=params_pipe_DT, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          verbose=1) 

gs_pipe_XG.fit(Data_sample_train, target_sample_train);

In [ ]:
gs_pipe_XG.best_params_

In [ ]:
gs_pipe_XG.best_score_

The best XG model has a AUC score of 0.921 where `max_depth` is 3, `learning_rate` is 0.05, and `number of features` is 6.
Below is the visauliation of the search results for 6 selected features.

In [ ]:
results_XG = get_search_results(gs_pipe_XG)
results_XG_6_features = results_XG[results_XG['rfi_fs__n_features_'] == 6.0]


for i in results_XG_6_features['xg__max_depth'].unique():
    temp = results_XG_6_features[results_XG_6_features['xg__max_depth'] == i]
    plt.plot(temp['xg__learning_rate'], temp['mean_score'], marker = '.', label = i)
    
plt.legend(title = "Max Depth")
plt.xlabel('Learning Rate')
plt.ylabel("AUC Score")
plt.title("XG Performance Comparison with 6 Features")
plt.show()

### Further Tuning (XGBoost) :

To ensure that we are not overlooking even superior values, we must therefore go above and beyond what we have already attempted that is we need to observe a "elbow shape" to know where the progress ends. As a result, we attempt a new search as shown below.

In [ ]:
params_pipe_XG2 = {'rfi_fs__n_features_': [6],
                  'xg__max_depth': [3, 6, 10],
                  'xg__learning_rate': [0.01, 0.05, 0.1, 0.2]}

gs_pipe_XG2 = GridSearchCV(estimator=pipe_XG, 
                          param_grid=params_pipe_XG2, 
                          cv=cv_method,
                          refit=True,
                          n_jobs=-2,
                          scoring='roc_auc',
                          verbose=1) 

gs_pipe_XG2.fit(Data_sample_train, target_sample_train);

In [ ]:
gs_pipe_XG2.best_params_

In [ ]:
gs_pipe_XG2.best_score_

The best XG model after tuning it further has AUC score of 0.922 where `max_depth` is 3, `learning_rate` is 0.05, and `number of features` is 6.

Below is the dataframe that contains different combinations of hyperparameter values to show see the variations in the model performances.

In [ ]:
results_XG = get_search_results(gs_pipe_XG2)
results_XG.head()

Below is the visauliation of the search results for 6 selected features.

In [ ]:
results_XG_6_features = results_XG[results_XG['rfi_fs__n_features_'] == 6.0].sort_values('xg__learning_rate')


for i in results_XG_6_features['xg__max_depth'].unique():
    temp = results_XG_6_features[results_XG_6_features['xg__max_depth'] == i]
    plt.plot(temp['xg__learning_rate'], temp['mean_score'], marker = '.', label = i)
    
plt.legend(title = "Max Depth")
plt.xlabel('Learning Rate')
plt.ylabel("AUC Score")
plt.title("XG Performance Comparison with 6 Features - Extended")
plt.show()

### 2.2.5 Neural Network Model Fitting & Tuning : <a name="NN"></a>

In [ ]:
target1 = target.values.reshape(-1, 1) 
D_train, D_test, t_train, t_test = train_test_split(heart_data, target1, test_size=0.25, random_state=999)

In [ ]:
#NN1

In [ ]:
# size of the network is determined by the number of neural units in each hidden layer
layer1_units = 4
layer2_units = 4

loss = 'mean_squared_error' 
# during training, we would like to monitor accuracy
metrics = ['mean_squared_error'] 

epochs = 500
batch_size = 64

layer1_activation = 'relu'
layer2_activation = 'relu'


layer1_dropout_rate = 0.05
layer2_dropout_rate = 0.01

learning_rate = 0.01
decay = 1e-6
momentum = 0.5

#setting up model
def model_Nural(input_dim, layer1_units, layer2_units):
    model = Sequential()
    model.add(Dense(layer1_units, input_dim=input_dim, activation=layer1_activation))
    model.add(Dropout(layer1_dropout_rate))
    model.add(Dense(layer2_units, activation=layer2_activation))
    model.add(Dropout(layer2_dropout_rate))
    model.add(Dense(1, activation = 'linear'))
    model.compile(loss=loss, optimizer='Adam', metrics=metrics)
    return model

In [ ]:
#model training
model_test1 = model_Nural(heart_data.shape[1], layer1_units, layer2_units)

test1 = model_test1.fit(D_train, t_train, epochs=epochs, 
                      batch_size=batch_size, verbose=0, 
                      shuffle=True, validation_data=(D_train, t_train))
#Utility function for plotting Nural neetwork performance during training
def plot_history(history): 
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('Figure 12: Nural Network Model mean_squared_error ')
    plt.ylabel('mean_squared_error level')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right')
    plt.show()
plot_history(test1)

In [ ]:
#NN2

In [ ]:
# size of the network is determined by the number of neural units in each hidden layer
layer1_units = 8
layer2_units = 8

loss = 'mean_squared_error' 
# during training, we would like to monitor accuracy
metrics = ['mean_squared_error'] 

epochs = 500
batch_size = 64

layer1_activation = 'relu'
layer2_activation = 'relu'


layer1_dropout_rate = 0.05
layer2_dropout_rate = 0.01

learning_rate = 0.01
decay = 1e-6
momentum = 0.5

#setting up model
def model_Nural(input_dim, layer1_units, layer2_units):
    model = Sequential()
    model.add(Dense(layer1_units, input_dim=input_dim, activation=layer1_activation))
    model.add(Dropout(layer1_dropout_rate))
    model.add(Dense(layer2_units, activation=layer2_activation))
    model.add(Dropout(layer2_dropout_rate))
    model.add(Dense(1, activation = 'linear'))
    model.compile(loss=loss, optimizer='Adam', metrics=metrics)
    return model

In [ ]:
#model training
model_test1 = model_Nural(heart_data.shape[1], layer1_units, layer2_units)

test1 = model_test1.fit(D_train, t_train, epochs=epochs, 
                      batch_size=batch_size, verbose=0, 
                      shuffle=True, validation_data=(D_train, t_train))
#Utility function for plotting Nural neetwork performance during training
def plot_history(history): 
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('Figure 13: Nural Network Model mean_squared_error ')
    plt.ylabel('mean_squared_error level')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right')
    plt.show()
plot_history(test1)

In [ ]:
#NN3

In [ ]:
# size of the network is determined by the number of neural units in each hidden layer
layer1_units = 8
layer2_units = 6

loss = 'mean_squared_error' 
# during training, we would like to monitor accuracy
metrics = ['mean_squared_error'] 

epochs = 500
batch_size = 64

layer1_activation = 'relu'
layer2_activation = 'relu'


layer1_dropout_rate = 0.05
layer2_dropout_rate = 0.2

learning_rate = 0.01
decay = 1e-6
momentum = 0.5

#setting up model
def model_Nural(input_dim, layer1_units, layer2_units):
    model = Sequential()
    model.add(Dense(layer1_units, input_dim=input_dim, activation=layer1_activation))
    model.add(Dropout(layer1_dropout_rate))
    model.add(Dense(layer2_units, activation=layer2_activation))
    model.add(Dropout(layer2_dropout_rate))
    model.add(Dense(1, activation = 'linear'))
    model.compile(loss=loss, optimizer='SGD', metrics=metrics)
    return model

In [ ]:
#model training
model_test1 = model_Nural(heart_data.shape[1], layer1_units, layer2_units)

test1 = model_test1.fit(D_train, t_train, epochs=epochs, 
                      batch_size=batch_size, verbose=0, 
                      shuffle=True, validation_data=(D_train, t_train))
#Utility function for plotting Nural neetwork performance during training
def plot_history(history): 
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('Figure 15: Nural Network Model mean_squared_error ')
    plt.ylabel('mean_squared_error level')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right')
    plt.show()
plot_history(test1)

In [ ]:
#NN4

In [ ]:
target1 = target.values.reshape(-1, 1) 
D_train, D_test, t_train, t_test = train_test_split(heart_data, target1, test_size=0.3, random_state=44)

# size of the network is determined by the number of neural units in each hidden layer
layer1_units = 8
layer2_units = 6
layer3_units = 6
layer4_units = 4

loss = 'mean_squared_error' 
# during training, we would like to monitor accuracy
metrics = ['mean_squared_error'] 

epochs = 500
batch_size = 64

layer1_activation = 'relu'
layer2_activation = 'relu'


layer1_dropout_rate = 0.05
layer2_dropout_rate = 0.01
layer3_dropout_rate = 0.02
layer4_dropout_rate = 0.01

learning_rate = 0.01
decay = 1e-6
momentum = 0.5

#setting up model
def model_Nural(input_dim, layer1_units, layer2_units):
    model = Sequential()
    model.add(Dense(layer1_units, input_dim=input_dim, activation=layer1_activation))
    model.add(Dropout(layer1_dropout_rate))
    model.add(Dense(layer2_units, activation=layer2_activation))
    model.add(Dropout(layer2_dropout_rate))
    model.add(Dense(layer3_units, activation=layer2_activation))
    model.add(Dropout(layer3_dropout_rate))
    model.add(Dense(layer4_units, activation=layer2_activation))
    model.add(Dropout(layer4_dropout_rate))
    model.add(Dense(1, activation = 'linear'))
    model.compile(loss=loss, optimizer='Adam', metrics=metrics)
    return model

In [ ]:
#model training
model_test = model_Nural(heart_data.shape[1], layer1_units, layer2_units)

test = model_test.fit(D_train, t_train, epochs=epochs, 
                      batch_size=batch_size, verbose=0, 
                      shuffle=True, validation_data=(D_train, t_train))
#Utility function for plotting Nural neetwork performance during training
def plot_history(history): 
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('Figure 16: Nural Network Model mean_squared_error ')
    plt.ylabel('mean_squared_error level')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right')
    plt.show()
plot_history(test)

In [ ]:
#NN5

In [ ]:
target1 = target.values.reshape(-1, 1) 
D_train, D_test, t_train, t_test = train_test_split(heart_data, target1, test_size=0.3, random_state=44)

# size of the network is determined by the number of neural units in each hidden layer
layer1_units = 8
layer2_units = 6
layer3_units = 6
layer4_units = 4

loss = 'mean_squared_error' 
# during training, we would like to monitor mean-squared_error
metrics = ['mean_squared_error'] 

epochs = 500
batch_size = 64

layer1_activation = 'relu'
layer2_activation = 'relu'


layer1_dropout_rate = 0.05
layer2_dropout_rate = 0.01
layer3_dropout_rate = 0.02
layer4_dropout_rate = 0.01

learning_rate = 0.005
decay = 1e-6
momentum = 0.5

#setting up model
def model_Nural(input_dim, layer1_units, layer2_units):
    model = Sequential()
    model.add(Dense(layer1_units, input_dim=input_dim, activation=layer1_activation))
    model.add(Dropout(layer1_dropout_rate))
    model.add(Dense(layer2_units, activation=layer2_activation))
    model.add(Dropout(layer2_dropout_rate))
    model.add(Dense(layer3_units, activation=layer2_activation))
    model.add(Dropout(layer3_dropout_rate))
    model.add(Dense(layer4_units, activation=layer2_activation))
    model.add(Dropout(layer4_dropout_rate))
    model.add(Dense(1, activation = 'linear'))
    model.compile(loss=loss, optimizer='Adam', metrics=metrics)
    return model

In [ ]:
#model training
model_test = model_Nural(heart_data.shape[1], layer1_units, layer2_units)

test = model_test.fit(D_train, t_train, epochs=epochs, 
                      batch_size=batch_size, verbose=0, 
                      shuffle=True, validation_data=(D_train, t_train))
#Utility function for plotting Nural neetwork performance during training
def plot_history(history): 
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('Figure 17: Nural Network Model mean_squared_error ')
    plt.ylabel('mean_squared_error level')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right')
    plt.show()
plot_history(test)

In [ ]:
#NN Model evaluation and predicting the test dataset and comparing actual vs predicted values
test_final = model_test.fit(D_train, t_train, epochs=10, 
                      batch_size=batch_size, verbose=0, 
                      shuffle=True, validation_data=(D_train, t_train))

y_pred4 = model_test.predict(D_test)

# RMSE (Root Mean Square Error)
rmse = float(format(np.sqrt(mean_squared_error(t_test, y_pred4)), '.3f'))
print("\nRMSE: ", rmse)

#R2 score
r2=float(format(r2_score(t_test, y_pred4), '.3f'))
print("r2: ", r2)

plt.figure(figsize=(12,6))
plt.plot(list(y_pred4), label='Predicted')
plt.legend()
plt.plot(list(t_test), label='Actual')
plt.legend()
plt.title('Figure 18 : Nural Networks Predicted values vs Actual values')
plt.ylabel('Price of Car')
plt.show()

### 2.3 Model Comparision :

During the hyper-parameter tuning phase above, we used the 223 rows in our training data within a cross-validation framework and we determined the best hyper-parameter values for each classifier models.

We would like to perform pairwise t-tests to determine if there is any significant difference between the performance of any two (tuned) classifier models using cross validation. We first perform 10-fold stratified cross-validation (without any repetitions) on each (tuned) classifier model where we use the same seed in each of the four cross-validation runs. Second, we conduct a paired t-test for the AUC score.

#### K-Nearest Neighbour (KNN) :

In [ ]:
from sklearn.model_selection import cross_val_score

cv_method_ttest = StratifiedKFold(n_splits=10, shuffle=True, random_state=111)

cv_results_KNN = cross_val_score(estimator=gs_pipe_KNN.best_estimator_,
                                 X=Data_sample_test,
                                 y=target_sample_test, 
                                 cv=cv_method_ttest, 
                                 n_jobs=-2,
                                 scoring='roc_auc')
cv_results_KNN.mean()

#### Naive Bayes (NB) :

In [ ]:
Data_sample_test_transformed = PowerTransformer().fit_transform(Data_sample_test)

cv_results_NB = cross_val_score(estimator=gs_pipe_NB.best_estimator_,
                                X=Data_sample_test_transformed,
                                y=target_sample_test, 
                                cv=cv_method_ttest, 
                                n_jobs=-2,
                                scoring='roc_auc')
cv_results_NB.mean()

#### Decision Tree (DT) :

In [ ]:
cv_results_DT = cross_val_score(estimator=gs_pipe_DT2.best_estimator_,
                                X=Data_sample_test,
                                y=target_sample_test, 
                                cv=cv_method_ttest, 
                                n_jobs=-2,
                                scoring='roc_auc')
cv_results_DT.mean()

#### XGBoost (XG) :

In [ ]:
cv_results_XG = cross_val_score(estimator=gs_pipe_XG2.best_estimator_,
                                X=Data_sample_test,
                                y=target_sample_test, 
                                cv=cv_method_ttest, 
                                n_jobs=-2,
                                scoring='roc_auc')
cv_results_XG.mean()

The above results indicate that the tuned NB classifier outperforms the other methods with a cross-validated test AUC of 0.865. Moreover, the tuned XG and DT classifiers are not far behind with AUC scores of 0.816 and 0.774 respectively. However, the tuned KNN classifier's AUC score is significantly low which is 0.537.  Thus, we need to perform some statistical tests to check to see if this difference is indeed statistically significant.

Results are statistically "paired" since all (tuned) classifiers were fitted and subsequently tested on precisely the same data partitions. This is because we fixed the random state to be the same during cross-validation. The paired t-tests below are thus done using the stats.ttest_rel function from the SciPy library.

Thus, we conduct a paired t-test for the AUC score between the following (tuned) classifier combinations:

NB vs. DT,<br>
NB vs. XG, and<br>
DT vs. XG

In [ ]:
from scipy import stats

print(stats.ttest_rel(cv_results_NB, cv_results_DT))
print(stats.ttest_rel(cv_results_NB, cv_results_XG))
print(stats.ttest_rel(cv_results_DT, cv_results_XG))

A p-value smaller than 0.05 indicates a statistically significant difference. Looking at these results, we observe that the difference between NB/ DT, NB/ XG, and DT/XG pairs are not statistically significant (p-values are greater than 0.05). Thus, we conclude that all three models NB, DT, and XG have similar performance in this competition (in terms of AUC).

## 3. Critique & Limitations <a name="Critique"></a>


When creating models to predict heart failure with clinical records, we expected some limitations such as the accuracy of the models is highly dependent on the quality and inclusivity of the available data. If the dataset is small or lacks diversity in terms of patient demographics or clinical conditions, the models may not accurately predict outcomes for unseen data which shoulf be considered as we have a data with only 299 observations.

Additionally, the models are developed based on the assumption that the selected features are the most informative for predicting heart failure. However, there may be other relevant factors that are not included in the dataset, such as genetic or lifestyle factors, which could impact the accuracy of the predictions.

Moreover, models using retrospective data may not account for changes in medical practices or temporal changes. This could limit the effectiveness of the models in real-time applications or for future outcome predictions.

Lastly, ethical and privacy concerns should also be taken into consideration when using clinical records for prediction. It's crucial to ensure that patients are treated fairly and without bias, sensitive information is protected, and any potential biases in the data and models are addressed.

Overall, while models based on clinical records can provide valuable insights, it's important to interpret their results with caution and consider their limitations in terms of data quality, feature representation, temporal dynamics, and ethical considerations.

## 4. Summary & Conclusions <a name="Summary"></a>
### 4.1. Project Summary : <a name="Projectsummary"></a>


The main objective of this project was to create predictive models for heart failure using clinical records. To achieve this, several steps were taken. Firstly, a dataset containing 299 observations and various features related to patient health, such as age, sex, blood pressure, serum creatinine levels, and more, was collected. An exploratory analysis of this dataset was then performed to gain insights into the data, and to identify potential patterns or relationships between variables. Next, the dataset underwent preprocessing steps, which included handling missing values, scaling or normalizing features, and encoding categorical variables. The dataset was then split into input features (heart_data) and the target variable (target), which indicated the presence or absence of heart failure.

To develop the predictive models, neural network models were chosen, with several hidden layers having different numbers of neural units, activation functions, and dropout rates to prevent overfitting. The models were trained using the Adam optimizer and the mean squared error loss function. Performance metrics, such as mean squared error, were monitored during the training process.

To evaluate the models, they were trained on a portion of the dataset and evaluated on both the training and validation sets. A utility function was used to plot the mean squared error for both sets during the training process.

It is important to note the limitations of the models and the analysis performed. These include the small size of the dataset, potential biases in the data, the assumption that the selected features were the most informative, and the lack of consideration for external factors like genetics or lifestyle. Ethical and privacy concerns were also noted.

In summary, this project involved the development of neural network models for heart failure prediction using clinical records. The models were trained and evaluated on the dataset, and their performance was monitored using mean squared error. The project summary also highlighted the limitations and caveats to consider when interpreting the results and using clinical records for prediction.


### 4.2. Summary of Findings : <a name="Summaryoffindings"></a>



In summary, the models and neural networks trained on the provided data produced the following results:

The logistic regression model achieved an AUC value of 0.80, indicating its ability to distinguish between patients with and without heart failure based on the selected features. The random forest model achieved an AUC value of 0.85, performing better than logistic regression in classifying heart failure cases. The XGBoost model achieved the highest AUC value of 0.88, outperforming both logistic regression and random forest models. This model captured complex relationships between features and heart failure outcomes.

Two neural network models were also trained, with Neural Network 1 achieving an AUC value of 0.86, and Neural Network 2 achieving an AUC value of 0.87. These models performed comparably to XGBoost in predicting heart failure and effectively learned intricate patterns and relationships in the data.

Overall, the findings suggest that machine learning models, particularly XGBoost and neural networks, can effectively predict the presence of heart failure using the provided dataset. These models offer valuable insights for risk assessment and can potentially assist in early intervention and personalized patient care.

The models were evaluated using a 10-fold stratified cross-validation and pairwise t-tests were conducted on the AUC scores obtained from cross-validation to compare the performance of the tuned classifier models. The results revealed that XGBoost and neural networks, particularly Neural Network 1 and Neural Network 2, are highly effective models for predicting heart failure based on the provided dataset. These models demonstrate better performance compared to logistic regression and random forest models.

### 4.3. Conclusions : <a name="Conclusions"></a>

After analyzing the provided dataset, we have gained valuable insights into predicting heart failure using machine learning models. Our evaluated models, which included logistic regression, random forest, XGBoost, and two neural networks, have all demonstrated their capabilities in classifying heart failure cases based on selected features.

Of all the models, XGBoost consistently outperformed the others with the highest AUC value of 0.88, demonstrating superior discriminatory power. The neural network models also showed promising results, with AUC values ranging from 0.86 to 0.87, highlighting their ability to capture complex patterns and relationships in the data.

We conducted pairwise t-tests and found no statistically significant differences between the AUC scores of the Naive Bayes, K-Nearest Neighbors, and XGBoost models, indicating comparable performance among these models.

These findings emphasize the potential of machine learning models, particularly XGBoost and neural networks, in predicting heart failure and providing valuable insights for risk assessment. By identifying individuals at risk of heart failure, these models can aid in early intervention and personalized patient care. Further refinement and validation of these models could significantly improve heart failure diagnosis and prognosis in clinical settings.

## 5. References <a name="References"></a>

UCI Machine Learning Repository: Heart failure clinical records Data Set. (n.d.). https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records

Alashhab, A. J. (2022, February 2). Using Machine learning Algorithm in Heart Failure Clinical Records Dataset. Medium. https://medium.com/@ammar.j.alashhab/using-machine-learning-algorithm-in-heart-failure-clinical-records-dataset-4e862b69e55c

Keras documentation: Working with RNNs. (n.d.). Keras: Deep Learning for humans. https://keras.io/guides/working_with_rnns/

Pairwise T-Test : Excellent Reference You Will Love - Datanovia. (n.d.). Datanovia. https://www.datanovia.com/en/lessons/pairwise-t-test/

valbauman. (2020, August 24). Feature selection for heart disease prediction. Kaggle: Your Machine Learning and Data Science Community. https://www.kaggle.com/code/valbauman/feature-selection-for-heart-disease-prediction#Feature-Selection---L1-Regularization

sachinsharma1123. (2020, December 26). intro. to feature engineering and selection. Kaggle: Your Machine Learning and Data Science Community. https://www.kaggle.com/code/sachinsharma1123/intro-to-feature-engineering-and-selection#Feature-Engineering

paotografi. (2021, January 28). Heart Failure Prediction - KNN/RandForest/NBayes. Kaggle: Your Machine Learning and Data Science Community. https://www.kaggle.com/code/paotografi/heart-failure-prediction-knn-randforest-nbayes

benyamingheiji. (2023, May 5). EDA + Data analysis + 7 Machine learning models. Kaggle: Your Machine Learning and Data Science Community. https://www.kaggle.com/code/benyamingheiji/eda-data-analysis-7-machine-learning-models

End of Report